In [ ]:
import copy
import optuna
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bcmix import *

In [ ]:
fed = pd.read_csv("realdata/FEDFUNDS.csv")
pce = pd.read_csv("realdata/EXPINF1YR.csv")

In [ ]:
X = fed["FEDFUNDS"].values
y = pce["EXPINF1YR"].values

In [ ]:
CP1 = 32
CP2 = 62
CP3 = 87
CP4 = 92
BETA = -0.68
ALPHA = np.full(len(y), 1.8234)
ALPHA[CP1:] = 3.1821
ALPHA[CP2:] = 1.7912
ALPHA[CP3:] = 4.4477
ALPHA[CP4:] = 5.8484

y_hat = ALPHA + BETA * X
res = y - y_hat
SIGMA = (res.T @ res / len(y)) ** 0.5
BETA /= SIGMA
ALPHA /= SIGMA

In [ ]:
N_TRIALS = 10
DATA_LEN = len(y)
P = 0.02

In [ ]:
mean_true = np.array([[3.33 / SIGMA], [-1.286]])
covm_true = np.array([[1.0, 0.0], [0.0, 1.0]])
# prior
canonical_0 = np.array([[3.33 / SIGMA], [-1.286]])
precision_0 = np.array([[1.0, 0.0], [0.0, 1.0]])
logcon_0 = (np.linalg.slogdet(precision_0)[1] - (canonical_0.T @ np.linalg.inv(precision_0) @ canonical_0).item()) / 2
states = {0: {"can": canonical_0, "pre": precision_0, "log": logcon_0, "pit": 0.0}}

In [ ]:
def objective_bcmix(trial):
    a = trial.suggest_float('a', ACTION_RANGE_BCMIX[0], ACTION_RANGE_BCMIX[1])
    q = q_myopic_with_change(states_i, a, P, xstar_bcmix)
    return q

def objective(trial):
    a = trial.suggest_float('a', ACTION_RANGE[0], ACTION_RANGE[1])
    q = q_myopic_without_change(canonical_i, precision_i, a, xstar)
    return q

In [ ]:
for i in range(1):
    # initialize
    simresult_bcmix_i = np.full((DATA_LEN, 6 * (1 + M1 + M2) + 4), 0.0)
    simresult_i = np.full((DATA_LEN, 9), np.nan)
    canonical_i, precision_i, states_i = canonical_0, precision_0, copy.deepcopy(states)
    xstar_bcmix = 0
    xstar = 0
    for j in range(DATA_LEN):
        alpha_i = ALPHA[j]
        beta_i = BETA
        # current state
        # bcmix model
        simresult_bcmix_i[j, 0] = alpha_i
        simresult_bcmix_i[j, 1] = beta_i
        for m, s in states_i.items():
            covm_bcmix_i = np.linalg.inv(s["pre"])
            mean_bcmix_i = covm_bcmix_i @ s["can"]
            simresult_bcmix_i[j, m * 6 + 2] = mean_bcmix_i[0][0]
            simresult_bcmix_i[j, m * 6 + 3] = mean_bcmix_i[1][0]
            simresult_bcmix_i[j, m * 6 + 4] = covm_bcmix_i[0][0]
            simresult_bcmix_i[j, m * 6 + 5] = covm_bcmix_i[0][1]
            simresult_bcmix_i[j, m * 6 + 6] = covm_bcmix_i[1][1]
            simresult_bcmix_i[j, m * 6 + 7] = s["pit"]
        # old model
        simresult_i[j, 0] = alpha_i
        simresult_i[j, 1] = beta_i
        covm_i = np.linalg.inv(precision_i)
        mean_i = covm_i @ canonical_i
        simresult_i[j, 2] = mean_i[0][0]
        simresult_i[j, 3] = mean_i[1][0]
        simresult_i[j, 4] = covm_i[0][0]
        simresult_i[j, 5] = covm_i[0][1]
        simresult_i[j, 6] = covm_i[1][1]
        # select action
        # bcmix model
        x_m_bcmix = myopic_mix(states_i, P, xstar_bcmix)[0]
        #ACTION_RANGE_BCMIX = (max(x_m_bcmix - 0.5, 0), max(x_m_bcmix + 0.5, 0.25))
        #study_bcmix = optuna.create_study(direction="maximize")
        #study_bcmix.optimize(objective_bcmix, n_trials=N_TRIALS)
        #a_bcmix = study_bcmix.best_trial.params['a']
        a_bcmix = max(x_m_bcmix, 0)
        simresult_bcmix_i[j, -2] = a_bcmix
        xstar_bcmix = a_bcmix
        # old model
        x_m = myopic(canonical_i, precision_i, xstar)[0]
        #ACTION_RANGE = (max(x_m - 0.5, 0), max(x_m + 0.5, 0.25))
        #study = optuna.create_study(direction="maximize")
        #study.optimize(objective, n_trials=N_TRIALS)
        #a = study.best_trial.params['a']
        a = max(x_m, 0)
        simresult_i[j, 7] = a
        xstar = a
        # update state
        # bcmix model
        y_bcmix = env_response(a_bcmix, alpha_i, beta_i, mean_true=None, covm_true=None, p=0, err=res[j])[0]
        simresult_bcmix_i[j, -1] = y_bcmix * SIGMA
        states_i = update_with_change(states_i, a_bcmix, y_bcmix, P)
        # old model
        y = env_response(a, alpha_i, beta_i, err=res[j])[0]
        simresult_i[j, 8] = y * SIGMA
        canonical_i, precision_i = update_without_change(canonical_i, precision_i, a, y)
    np.save("realdata\inf_cgbcmix_w01_" + str(i) + ".npy", simresult_bcmix_i)
    np.save("realdata\inf_cgold_w01_" + str(i) + ".npy", simresult_i)

In [ ]:
simresult_w01 = np.zeros(DATA_LEN)
simresult_w10 = np.zeros(DATA_LEN)

for i in range(1):
    simresult_w01 += np.load("realdata\inf_cgbcmix_w01_" + str(i) + ".npy")[:, -2]
    simresult_w10 += np.load("realdata\inf_cgbcmix_w10_" + str(i) + ".npy")[:, -2]

simresult_w01 /= 1
simresult_w10 /= 1

plt.figure(figsize=(7.2, 4.8))
plt.plot(fed["observation_date"], simresult_w01, "-", color="grey", label="w=0.1")
plt.plot(fed["observation_date"], simresult_w10, "k-", label="w=1")
plt.plot(fed["observation_date"], fed["FEDFUNDS"].values, "k:", label="true")
plt.xticks(fed["observation_date"].iloc[[0, 12, 24, 36, 48, 60, 72, 84, 96, 108, 120]], rotation=45, fontsize=6)
plt.legend(loc="upper left")
plt.title("federal funds effective rate")
plt.ylabel("interest rate")
#plt.savefig("fig3", dpi=300)

In [ ]:
simresult_w01 = np.zeros(DATA_LEN)
simresult_w10 = np.zeros(DATA_LEN)

for i in range(1):
    simresult_w01 += np.load("realdata\inf_cgbcmix_w01_" + str(i) + ".npy")[:, -1]
    simresult_w10 += np.load("realdata\inf_cgbcmix_w10_" + str(i) + ".npy")[:, -1]

simresult_w01 /= 1
simresult_w10 /= 1

plt.figure(figsize=(7.2, 4.8))
plt.plot(pce["observation_date"], simresult_w01, "-", color="grey", label="w=0.1")
plt.plot(pce["observation_date"], simresult_w10, "k-", label="w=1")
plt.plot(pce["observation_date"], pce["EXPINF1YR"].values, "k:", label="true")
plt.xticks(pce["observation_date"].iloc[[0, 12, 24, 36, 48, 60, 72, 84, 96, 108, 120]], rotation=45, fontsize=6)
plt.legend(loc="upper left")
plt.title("1-year expected inflation")
plt.ylabel("expected inflation")
#plt.savefig("fig4", dpi=300)